In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[8],6
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,5e-25
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[4]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`              
            layer    coolrg                layer        coolrg
pressure                                                      
0.000312        1 -0.000020                    1  1.843733e-06
0.000750        2 -0.000019                    2  2.567441e-05
0.001052        3 -0.000019                    3 -1.128661e-05
0.001476        4 -0.000019                    4 -1.056360e-05
0.002070        5 -0.000018                    5 -1.214943e-05
0.002904        6 -0.000018                    6 -1.100636e-05
0.004074        7 -0.000017                    7 -1.168221e-05
0.005714        8 -0.000016                    8 -1.170574e-05
0.008015        9 -0.000016                    9 -1.139949e-05
0.011243       10 -0.000015                   10 -1.158150e-05
0.015771       11 -0.000014                   11 -1.130817e-05
0.022122       12 -0.000014                   12 -1.138963e-05
0.031031       13 -0.000013                   13 -1.055468e-05
0.043528       14 -0.000012                   14 -5.787599e-06
0.061057       15 -0.000007                   15 -3.958634e-06
0.085645       16  0.000005                   16 -4.658945e-06
0.120136       17  0.000028                   17 -1.310197e-06
0.168516       18  0.000062                   18  6.296597e-06
0.236378       19  0.000113                   19  1.806361e-05
0.331549       20  0.000185                   20  3.535745e-05
0.465100       21  0.000285                   21  5.976112e-05
0.652400       22  0.000408                   22  8.941392e-05
0.915100       23  0.000456                   23  1.030832e-04
1.283650       24  0.000408                   24  9.258540e-05
1.800600       25  0.000331                   25  7.373292e-05
2.525700       26  0.000243                   26  5.212334e-05
3.542800       27  0.000170                   27  3.382779e-05
4.969550       28  0.000114                   28  1.972628e-05
6.970850       29  0.000072                   29  8.976696e-06
9.778100       30  0.000041                   30  8.207875e-07
13.715850      31  0.000018                   31 -5.319316e-06
19.239350      32  0.000001                   32 -9.943513e-06
26.987250      33 -0.000010                   33 -1.343342e-05
37.855300      34 -0.000018                   34 -1.609318e-05
53.100050      35 -0.000024                   35 -1.848348e-05
73.887500      36 -0.000027                   36 -1.993788e-05
97.662500      37 -0.000027                   37 -2.066632e-05
121.437500     38 -0.000028                   38 -2.263784e-05
145.212500     39 -0.000030                   39 -2.547277e-05
168.987500     40 -0.000033                   40 -2.978024e-05
192.762500     41 -0.000042                   41 -4.394519e-05
216.537500     42 -0.000071                   42 -9.395490e-05
240.312500     43 -0.000071                   43 -1.584575e-04
264.087500     44  0.000021                   44 -2.024840e-04
287.862500     45  0.000289                   45 -1.507649e-04
311.637500     46  0.000815                   46  1.168303e-04
335.412500     47  0.001649                   47  7.357199e-04
359.187500     48  0.002777                   48  1.807717e-03
382.962500     49  0.004037                   49  3.196118e-03
406.737500     50  0.005660                   50  4.921523e-03
430.512500     51  0.007695                   51  6.881844e-03
454.287500     52  0.010193                   52  9.009498e-03
478.062500     53  0.013196                   53  1.133506e-02
501.837500     54  0.016716                   54  1.394968e-02
525.612500     55  0.020812                   55  1.721554e-02
549.387500     56  0.025490                   56  2.144756e-02
573.162500     57  0.030794                   57  2.695501e-02
596.937500     58  0.036796                   58  3.373883e-02
620.712500     59  0.043533                   59  4.164159e-02
644.487500     60  0.051121                   60  5.043927e-02
668.262500     61  0.059515                  

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -23.814720  0.000000e+00 -23.814720 -23.925324   
0.000624    2     -23.814720  1.140301e-09 -23.814720 -23.925324   
0.000876    3     -23.814720  1.608101e-09 -23.814720 -23.925324   
0.001229    4     -23.814720  2.289092e-09 -23.814720 -23.925324   
0.001723    5     -23.814720  3.281216e-09 -23.814720 -23.925324   
0.002417    6     -23.814720  4.726189e-09 -23.814720 -23.925324   
0.003391    7     -23.814720  6.830919e-09 -23.814720 -23.925324   
0.004757    8     -23.814720  9.896017e-09 -23.814720 -23.925324   
0.006672    9     -23.814720  1.435888e-08 -23.814720 -23.925324   
0.009359    10    -23.814720  2.085623e-08 -23.814720 -23.925324   
0.013128    11    -23.814720  3.031421e-08 -23.814720 -23.925324   
0.018415    12    -23.814720  4.408038e-08 -23.814720 -23.925324   
0.025830    13    -23.814720  6.411466e-08 -23.814720 -23.925324   
0.036232    14    -23.814720  9.327152e-08 -23.814720 -23.925324   
0.050823    15    -23.814720  1.361291e-07 -23.814720 -23.925324   
0.071291    16    -23.814720  2.068440e-07 -23.814720 -23.925324   
0.100000    17    -23.814720  3.427695e-07 -23.814720 -23.925324   
0.140271    18    -23.814721  6.155818e-07 -23.814720 -23.925324   
0.196760    19    -23.814721  1.158364e-06 -23.814720 -23.925324   
0.275997    20    -23.814721  2.227689e-06 -23.814719 -23.925324   
0.387100    21    -23.814720  4.313025e-06 -23.814716 -23.925324   
0.543100    22    -23.814719  8.343209e-06 -23.814711 -23.925324   
0.761700    23    -23.814716  1.580980e-05 -23.814700 -23.925324   
1.068500    24    -23.814711  2.725315e-05 -23.814684 -23.925323   
1.498800    25    -23.814705  4.182765e-05 -23.814663 -23.925323   
2.102400    26    -23.814698  5.902252e-05 -23.814639 -23.925323   
2.949000    27    -23.814693  7.793443e-05 -23.814615 -23.925323   
4.136600    28    -23.814689  9.822661e-05 -23.814591 -23.925325   
5.802500    29    -23.814688  1.198450e-04 -23.814568 -23.925328   
8.139200    30    -23.814691  1.427104e-04 -23.814548 -23.925333   
11.417000   31    -23.814699  1.667748e-04 -23.814532 -23.925342   
16.014700   32    -23.814714  1.920053e-04 -23.814522 -23.925354   
22.464000   33    -23.814739  2.183918e-04 -23.814521 -23.925372   
31.510500   34    -23.814778  2.459715e-04 -23.814532 -23.925399   
44.200100   35    -23.814834  2.745078e-04 -23.814560 -23.925436   
62.000000   36    -23.814911  3.006322e-04 -23.814610 -23.925489   
85.775000   37    -23.815005  3.206494e-04 -23.814685 -23.925556   
109.550000  38    -23.815096  3.351569e-04 -23.814761 -23.925624   
133.325000  39    -23.815195  3.543847e-04 -23.814841 -23.925700   
157.100000  40    -23.815315  3.887218e-04 -23.814926 -23.925792   
180.875000  41    -23.815469  4.496725e-04 -23.815019 -23.925911   
204.650000  42    -23.815722  5.835705e-04 -23.815138 -23.926112   
228.425000  43    -23.816349  1.011232e-03 -23.815338 -23.926624   
252.200000  44    -23.817680  2.142456e-03 -23.815537 -23.927745   
275.975000  45    -23.820188  4.708476e-03 -23.815480 -23.929909   
299.750000  46    -23.824491  9.824211e-03 -23.814667 -23.933702   
323.525000  47    -23.831338  1.896562e-02 -23.812372 -23.939843   
347.300000  48    -23.841599  3.387120e-02 -23.807729 -23.949159   
371.075000  49    -23.856068  5.616083e-02 -23.799906 -23.962407   
394.850000  50    -23.874517  8.598151e-02 -23.788536 -23.979407   
418.625000  51    -23.897482  1.248853e-01 -23.772596 -24.000664   
442.400000  52    -23.925463  1.745382e-01 -23.750925 -24.026648   
466.175000  53    -23.958921  2.367040e-01 -23.722218 -24.057838   
489.950000  54    -23.998276  3.132220e-01 -23.685054 -24.094635   
513.725000  55    -24.043808  4.058305e-01 -23.637977 -24.137369   
537.500000  56    -24.095785  5.164215e-01 -23.579364 -24.186232   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')